# Nitrite Dynamics

In this interactive document, I will introduce the consumer-resource models (CRM) for biomass growth and denitrification.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
import ipywidgets as widgets

## CRM for nitrate dynamics

First, let us only consider the nitrate ($A$, unit $mM$) dynamics, and the biomass ($x$, unit $g$) growth due to nitrate consumption. The CRM is the following:
$$ \dot{A}=-r_Ax\frac{A}{A+K_A} $$
$$ \dot{x} = x\gamma_A\frac{A}{A+K_A} $$

The parameters in the equation are:
- $r_A$: the per biomass consumption rate of $A$, unit $mM/(g\cdot day)$.
- $K_A$: the affinity constant to $A$, unit $mM$.
- $\gamma_A$: the growth rate of $x$, unit $day^{-1}$. (For CHL+ control, $\gamma_A$=0.)

We can combine the parameters to define a new variable $R_A=r_Ax$, representing the total consumption rate (unit $mM/day$).

Here, $H_A(A)=\frac{A}{A+K_A}$ is the Monod function. In the following discussion, we assume the affinity $K_A<<A$. So the Monod function is degenerated to the step function:
$$H_A(A=0)=0$$
$$H_A(A>0)=1$$
We will use the step function in the simulations. For the reason of normalization, we set $r_A=1$. We set $A(0)=2mM$ based on our experiment setup. Therefore, the dynamics will depend on $\gamma_A$, and $x(0)$.

In [2]:
t = np.linspace(0, 4, num=4000)
dt = 0.001

def f0(x0,ga):
    a = np.ones(len(t))*2
    x = np.ones(len(t))*x0
    ac = np.ones(len(t))*2
    xc = np.ones(len(t))*x0
    for ii in range(len(t)-1):
        a[ii+1] = np.maximum(a[ii]-x[ii]*dt,0)
        x[ii+1] = x[ii]+x[ii]*ga*dt*(a[ii]>0)
        ac[ii+1] = np.maximum(ac[ii]-xc[ii]*dt,0)
    plt.subplot(2,2,1)
    plt.plot(t, a, color='tab:blue')
    plt.plot(t, ac, color='tab:blue', linestyle='dashed')
    plt.ylim(-0.2, 2.2)
    plt.ylabel('metabolite')
    plt.subplot(2,2,3)
    plt.plot(t, x, color='tab:blue')
    plt.plot(t, xc, color='tab:blue', linestyle='dashed')
    plt.yscale('log')
    plt.ylim(0.01,100)
    plt.ylabel('biomass')
    plt.show()
    
nitrate_plot = interactive(
    f0, 
    x0=widgets.FloatSlider(min=0.01,max=1,step=0.01,value = 0.2,description = r'\(x(0)\)'),
    ga=widgets.FloatSlider(min=0,max=4,step=0.01,value = 1.0,description = r'\(\gamma_A\)'), 
)
nitrate_plot

interactive(children=(FloatSlider(value=0.2, description='\\(x(0)\\)', max=1.0, min=0.01, step=0.01), FloatSli…

## CRM for nitrate dynamics with carbon limitation

Let us only consider the nitrate ($A$, unit $mM$) dynamics, and the biomass ($x$, unit $g$) growth due to nitrate consumption, but limited by the amount of carbon (C). The CRM is the following:
$$ \dot{A}=-r_AxH_A(A) $$
$$ \dot{C}=-r_CxH_C(C) $$
$$ \dot{x} = x\gamma_AH_A(A)H_C(C) $$

Similarly, the $H_C(C)$ is the Monod function of $C$, $r_C$ is the consumption rate of $C$.

In the simulations, we set $r_C=1$ for the reason of normalization. Then the dynamics will depend on $\gamma_A$, $x(0)$ and $C(0)$.

In [3]:
def f0(ga,x0,c0):
    a = np.ones(len(t))*2
    c = np.ones(len(t))*c0
    x = np.ones(len(t))*x0
    ac = np.ones(len(t))*2
    cc = np.ones(len(t))*c0
    xc = np.ones(len(t))*x0
    for ii in range(len(t)-1):
        a[ii+1] = np.maximum(a[ii]-x[ii]*dt,0)
        c[ii+1] = np.maximum(c[ii]-x[ii]*dt,0)
        x[ii+1] = x[ii]+x[ii]*ga*dt*(a[ii]>0)*(c[ii]>0)
        ac[ii+1] = np.maximum(ac[ii]-xc[ii]*dt,0)
        cc[ii+1] = np.maximum(cc[ii]-xc[ii]*dt,0)
    plt.subplot(2,2,1)
    plt.plot(t, a, color='tab:blue')
    plt.plot(t, ac, color='tab:blue', linestyle='dashed')
    plt.ylim(-0.2, 2.2)
    plt.ylabel('metabolite')
    plt.subplot(2,2,2)
    plt.plot(t, c, color='tab:purple')
    plt.plot(t, cc, color='tab:purple', linestyle='dashed')
    plt.ylim(-0.2, 2.2)
    plt.title('carbon')
    plt.subplot(2,2,3)
    plt.plot(t, x, color='tab:blue')
    plt.plot(t, xc, color='tab:blue', linestyle='dashed')
    plt.yscale('log')
    plt.ylim(0.01,100)
    plt.ylabel('biomass')
    plt.show()
    
nitrate_plot2 = interactive(
    f0, 
    x0=widgets.FloatSlider(min=0.01,max=1,step=0.01,value = 0.2,description = r'\(x(0)\)'),
    c0=widgets.FloatSlider(min=0,max=2.2,step=0.01,value = 0.5,description = r'\(C(0)\)'), 
    ga=widgets.FloatSlider(min=0,max=4,step=0.01,value = 1.0,description = r'\(\gamma_A\)'), 
)
nitrate_plot2

interactive(children=(FloatSlider(value=1.0, description='\\(\\gamma_A\\)', max=4.0, step=0.01), FloatSlider(v…

## CRM for nitrate/nitrite dynamics

Let us now model on the nitrate and nitrite ($I$) dynamics, without carbon limitation. First, let us consider the two biomass: the nitrate specialist ($x_A$) whose growth is due to $A$ consumption, and nitrite spectialist ($x_I$) whose growth is due to $I$ consumption. The equation is:
$$ \dot{A}=-r_Ax_AH_A(A) $$
$$ \dot{I}=r_Ax_AH_A(A)-r_Ix_IH_I(I) $$
$$ \dot{x}_A = x_A\gamma_AH_A(A) $$
$$ \dot{x}_I = x_I\gamma_IH_I(A) $$

Similarly, $r_I$ is per consumption rate of $I$, $\gamma_I$ is the growth rate of $x_I$, and $H_I$ is the Monod funtion for $I$.

In the simulation, we set $r_A=1$ and $r_I=1$ for normalization reason. The initial condition of nitrite is $I(0)=0$. Therefore, the dynamics will depend on $x_A(0)$, $x_I(0)$, $\gamma_A$ and $\gamma_I$.

In [4]:
def f0(xa0,xi0,ga,gi):
    a = np.ones(len(t))*2
    i = np.ones(len(t))*0
    xa = np.ones(len(t))*xa0
    xi = np.ones(len(t))*xi0
    ac = np.ones(len(t))*2
    ic = np.ones(len(t))*0
    xac = np.ones(len(t))*xa0
    xic = np.ones(len(t))*xi0
    for ii in range(len(t)-1):
        a[ii+1] = np.maximum(a[ii]-xa[ii]*dt,0)
        xa[ii+1] = xa[ii]+xa[ii]*ga*dt*(a[ii]>0)
        i[ii+1] = np.maximum(i[ii]+a[ii]-a[ii+1]-xi[ii]*dt,0)
        xi[ii+1] = xi[ii]+xi[ii]*gi*dt*(i[ii]>0)
        ac[ii+1] = np.maximum(ac[ii]-xac[ii]*dt,0)
        ic[ii+1] = np.maximum(ic[ii]+ac[ii]-ac[ii+1]-xic[ii]*dt,0)
    plt.subplot(2,2,1)
    plt.plot(t, a, color='tab:blue')
    plt.plot(t, ac, color='tab:blue', linestyle='dashed')
    plt.plot(t, i, color='tab:orange')
    plt.plot(t, ic, color='tab:orange', linestyle='dashed')
    plt.ylim(-0.2, 2.2)
    plt.ylabel('metabolite')
    plt.subplot(2,2,3)
    plt.plot(t, xa, color='tab:blue')
    plt.plot(t, xac, color='tab:blue', linestyle='dashed')
    plt.plot(t, xi, color='tab:orange')
    plt.plot(t, xic, color='tab:orange', linestyle='dashed')
    plt.yscale('log')
    plt.ylim(0.01,100)
    plt.ylabel('biomass')
    plt.show()
    
nitrite_plot = interactive(
    f0, 
    xa0=widgets.FloatSlider(min=0.01,max=1,step=0.01,value = 0.3,description = r'\(x_A(0)\)'),
    xi0=widgets.FloatSlider(min=0.01,max=1,step=0.01,value = 0.15,description = r'\(x_I(0)\)'),
    ga=widgets.FloatSlider(min=0,max=4,step=0.01,value = 1.0,description = r'\(\gamma_A\)'), 
    gi=widgets.FloatSlider(min=0,max=4,step=0.01,value = 1.5,description = r'\(\gamma_I\)'), 
)
nitrite_plot

interactive(children=(FloatSlider(value=0.3, description='\\(x_A(0)\\)', max=1.0, min=0.01, step=0.01), FloatS…

## CRM for nitrate/nitrite dynamics with carbon limitation

Let us now model on the nitrate and nitrite dynamics with the carbon limitation. We still consider the two biomass model, and both biomass are limited by the amount of carbon. The equation is:
$$ \dot{A}=-r_Ax_AH_A(A) $$
$$ \dot{I}=r_Ax_AH_A(A)-r_Ix_IH_I(I) $$
$$ \dot{C}=-r_C(x_A+x_I)H_C(C) $$
$$ \dot{x}_A = x_A\gamma_AH_A(A)H_C(C) $$
$$ \dot{x}_I = x_I\gamma_IH_I(A)H_C(C) $$

Therefore, the dynamics will depend on $x_A(0)$, $x_I(0)$, $C(0)$, $\gamma_A$ and $\gamma_I$.

In [5]:
def f0(xa0,xi0,c0,ga,gi):
    a = np.ones(len(t))*2
    i = np.ones(len(t))*0
    c = np.ones(len(t))*c0
    xa = np.ones(len(t))*xa0
    xi = np.ones(len(t))*xi0
    ac = np.ones(len(t))*2
    ic = np.ones(len(t))*0
    cc = np.ones(len(t))*c0
    xac = np.ones(len(t))*xa0
    xic = np.ones(len(t))*xi0
    for ii in range(len(t)-1):
        a[ii+1] = np.maximum(a[ii]-xa[ii]*dt,0)
        i[ii+1] = np.maximum(i[ii]+a[ii]-a[ii+1]-xi[ii]*dt,0)
        c[ii+1] = np.maximum(c[ii]-(xa[ii]+xi[ii])*dt,0)
        xa[ii+1] = xa[ii]+xa[ii]*ga*dt*(a[ii]>0)*(c[ii]>0)
        xi[ii+1] = xi[ii]+xi[ii]*gi*dt*(i[ii]>0)*(c[ii]>0)
        ac[ii+1] = np.maximum(ac[ii]-xac[ii]*dt,0)
        ic[ii+1] = np.maximum(ic[ii]+ac[ii]-ac[ii+1]-xic[ii]*dt,0)
        cc[ii+1] = np.maximum(cc[ii]-(xac[ii]+xic[ii])*dt,0)
    plt.subplot(2,2,1)
    plt.plot(t, a, color='tab:blue')
    plt.plot(t, ac, color='tab:blue', linestyle='dashed')
    plt.plot(t, i, color='tab:orange')
    plt.plot(t, ic, color='tab:orange', linestyle='dashed')
    plt.ylim(-0.2, 2.2)
    plt.ylabel('metabolite')
    plt.subplot(2,2,2)
    plt.plot(t, c, color='tab:purple')
    plt.plot(t, cc, color='tab:purple', linestyle='dashed')
    plt.ylim(-0.2, 2.2)
    plt.title('carbon')
    plt.subplot(2,2,3)
    plt.plot(t, xa, color='tab:blue')
    plt.plot(t, xac, color='tab:blue', linestyle='dashed')
    plt.plot(t, xi, color='tab:orange')
    plt.plot(t, xic, color='tab:orange', linestyle='dashed')
    plt.yscale('log')
    plt.ylim(0.01,100)
    plt.ylabel('biomass')
    plt.show()
    
nitrite_plot2 = interactive(
    f0, 
    xa0=widgets.FloatSlider(min=0.01,max=1,step=0.01,value = 0.3,description = r'\(x_A(0)\)'),
    xi0=widgets.FloatSlider(min=0.01,max=1,step=0.01,value = 0.15,description = r'\(x_I(0)\)'),
    c0=widgets.FloatSlider(min=0,max=2.2,step=0.01,value = 1.0,description = r'\(C(0)\)'), 
    ga=widgets.FloatSlider(min=0,max=4,step=0.01,value = 1.0,description = r'\(\gamma_A\)'), 
    gi=widgets.FloatSlider(min=0,max=4,step=0.01,value = 2.0,description = r'\(\gamma_I\)'), 
)
nitrite_plot2

interactive(children=(FloatSlider(value=0.3, description='\\(x_A(0)\\)', max=1.0, min=0.01, step=0.01), FloatS…